# ACS2 in Multiplexer

In [1]:
%matplotlib inline

# General
from __future__ import unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Logger
import logging
logging.basicConfig(level=logging.WARN)

# ALCS + Maze environment
import sys
sys.path.append('/Users/khozzy/Projects/pyalcs')
sys.path.append("/Users/khozzy/Projects/openai-maze-envs")

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS module
from alcs import ACS2, ACS2Configuration

# Load environments
import gym
import gym_multiplexer

## Multiplexer

In [2]:
mp = gym.make('boolean-multiplexer-6bit-v0')
situation = mp.reset()
mp.render()

'0101100'

In [3]:
state, reward, done, _ = mp.step(0)

print("New state: {}, reward: {}, is done: {}".format(state, reward, done))

New state: 0101100, reward: 0, is done: True


In [4]:
mp.render()

'0101100'

### Evaluation
Calculate knowledge - analyze all **reliable** classifiers and see what percentage of them makes proper classification

In [5]:
from gym_multiplexer.utils import get_correct_answer

def calculate_knowledge(env, population, ctrl_bits):
    p1 = env.render()  # state after executing action
    p0 = p1[:-1] + '0'  # initial state
    correct_answer = get_correct_answer(p0, ctrl_bits)  # true action
    
    reliable_classifiers = [c for c in population if c.is_reliable() and c.condition.does_match(p0)]
    nr_correct = sum([1 for cl in reliable_classifiers if cl.predicts_successfully(p0, correct_answer, p1)])
    
    return nr_correct / len(reliable_classifiers) * 100.0

## Go agent, go...

In [11]:
cfg = ACS2Configuration(mp.env.observation_space.n, 2, do_ga=True)
agent = ACS2(cfg)

population, metrics = agent.explore(mp, 10000)

In [13]:
print("Population size: {}".format(metrics[-1]['agent']['population']))
print("Reliable size: {}".format(metrics[-1]['agent']['reliable']))
print("Knowledge: {:.2f}%".format(calculate_knowledge(mp, population, 2)))
print(metrics[-1])

Population size: 454
Reliable size: 297
Knowledge: 0.00%
{'agent': {'population': 454, 'numerosity': 454, 'reliable': 297, 'fitness': 454.34897965995276, 'trial': 9999, 'steps': 1, 'total_steps': 10000}, 'environment': None}


In [14]:
p1 = mp.render()
p0 = p1[:-1] + '0'
correct_answer = get_correct_answer(p0, 2)

reliable_classifiers = [c for c in population if c.is_reliable() and c.condition.does_match(p0)]
for cl in reliable_classifiers:
    print("{}, q: {:.2f}, r: {:.2f}".format(cl, cl.q, cl.r))

01#0000-0-######1 @ 0x113c79cc0, q: 1.00, r: 999.98
01100#0-0-######1 @ 0x113cb37b8, q: 1.00, r: 999.99
0110###-1-####### @ 0x113d666d8, q: 1.00, r: 0.11
0110###-1-####### @ 0x113d66cc0, q: 1.00, r: 0.19
01#00##-0-######1 @ 0x113d66160, q: 1.00, r: 1000.00
#1#0#0#-1-####### @ 0x113cb3f98, q: 1.00, r: 0.00
#1#0#0#-0-######1 @ 0x113c0d908, q: 1.00, r: 1000.00
01#0###-1-####### @ 0x113c79e48, q: 1.00, r: 0.00
01#0###-1-####### @ 0x113d66d68, q: 1.00, r: 0.02
#1#000#-0-######1 @ 0x113c0d710, q: 1.00, r: 1000.00
01#0###-1-####### @ 0x113d66550, q: 1.00, r: 0.00
01#0#00-0-######1 @ 0x113ce7630, q: 1.00, r: 1000.00
01#0###-0-######1 @ 0x113d66278, q: 1.00, r: 1000.00
01#0###-1-####### @ 0x113d24d30, q: 0.99, r: 0.02
01#0#0#-1-####### @ 0x1136a5f28, q: 0.95, r: 0.87
0110##0-0-######1 @ 0x113ce7b70, q: 1.00, r: 993.84
01#0###-1-####### @ 0x113cb3208, q: 0.98, r: 0.02
#1#0#0#-0-######1 @ 0x113c0d470, q: 1.00, r: 1000.00
01#00##-0-######1 @ 0x113cb3dd8, q: 0.97, r: 1000.00
01#0###-1-####### @ 0x1

In [9]:
print("p0: {}".format(p0))
print("p1: {}".format(p1))
print("correct: {}".format(correct_answer))

p0: 0000110
p1: 0000111
correct: 0


In [10]:
nr_correct = sum([1 for cl in reliable_classifiers if cl.predicts_successfully(p0, correct_answer, p1) and c.condition.does_match(p0)])
nr_correct

NameError: name 'c' is not defined